# 1. Dataset

In [ ]:
# https://www.rondhuit.com/download.html#ldcc
# wget https://www.rondhuit.com/download/ldcc-20140209.tar.gz
# tar -zxvf ldcc-20140209.tar.gz


# 下記のプログラムでは、ニュースの各ファイルの本文を抽出し、改行、全角スペース、タブを除去して、1行の文に変換したのちに、カテゴリの番号（0-9）とのセットにします。


import glob
import os

raw_data_path = "./text"  # ライブドアニュースを格納したディレクトリ

dir_files = os.listdir(path=raw_data_path)
dirs = [f for f in dir_files if os.path.isdir(os.path.join(raw_data_path, f))]
text_label_data = []  # 文章とラベル（カテゴリ）のセット

for i in range(len(dirs)):
    dir = dirs[i]
    files = glob.glob(os.path.join(raw_data_path, dir, "*.txt"))

    for file in files:
        if os.path.basename(file) == "LICENSE.txt": # 各ディレクトリにあるLICENSE.txtを除外する
            continue

        with open(file, "r") as f:
            text = f.readlines()[3:]
            text = "".join(text)
            text = text.translate(str.maketrans({"\n":"", "\t":"", "\r":"", "\u3000":""})) 
            text_label_data.append([text, i])

In [ ]:
# 学習用、テスト用データの作成、保存
# 先ほど作成した、本文、ラベル（カテゴリ）のセットを、学習用、評価用のデータに分割し、CSVファイル（news_train.csv、news_test.csv）として保存します。

import csv
from sklearn.model_selection import train_test_split

news_train, news_test =  train_test_split(text_label_data, shuffle=True)  # データを学習用とテスト用に分割
data_path = "./data"

with open(os.path.join(data_path, "news_train.csv"), "w") as f:
  writer = csv.writer(f)
  writer.writerows(news_train) # 

with open(os.path.join(data_path, "news_test.csv"), "w") as f:
  writer = csv.writer(f)
  writer.writerows(news_test)

# 2. Training

In [ ]:
# 先ほど作成したデータを入力とし、BERTを使ってニュースのカテゴリ分類を学習（fine-tuning）させるプログラムtrain.pyを作成します。

# モデル、トークナイザーの読み込み
# transformersに含まれている文章を分類するためのモデルBertForSequenceClassification、日本語を形態素解析するためのトークナイザーBertJapaneseTokenizerを読み込みます。

# cl-tohoku/bert-base-japanese-whole-word-maskingは事前学習済みの日本語BERTモデルです。
# このモデルは、東北大学の乾研究室によって作成されたもので、こちらのページで公開されています。
# https://www.nlp.ecei.tohoku.ac.jp/news-release/3284/

# BertForSequenceClassification、BertJapaneseTokenizerを読み込んだ際に、自動的にダウンロードされるため、あらかじめダウンロードをする必要はありません。


from transformers import BertForSequenceClassification, BertJapaneseTokenizer

# モデル
model = BertForSequenceClassification.from_pretrained("cl-tohoku/bert-base-japanese-whole-word-masking", num_labels=9)
# model.cuda() # cudaを使う場合は、この行を有効にする
# トークナイザー
tokenizer = BertJapaneseTokenizer.from_pretrained("cl-tohoku/bert-base-japanese-whole-word-masking")



"""
学習用、テスト用データの読み込み
"""
# 学習の入力データとして、先ほど保存したデータを読み込みます。

import os
from datasets import load_dataset

# トークナイズ用関数
def tokenize(batch):
    return tokenizer(batch["text"], padding=True, truncation=True, max_length=128)
    
data_path = "./data"

# 学習用データ
train_data = load_dataset("csv", data_files=os.path.join(data_path, "news_train.csv"), column_names=["text", "label"], split="train")
train_data = train_data.map(tokenize, batched=True, batch_size=len(train_data))
train_data.set_format("torch", columns=["input_ids", "label"])

# テスト用データ
test_data = load_dataset("csv", data_files=os.path.join(data_path, "news_test.csv"), column_names=["text", "label"], split="train")
test_data = test_data.map(tokenize, batched=True, batch_size=len(test_data))
test_data.set_format("torch", columns=["input_ids", "label"]) 



"""
Trainerの初期化
"""
# Trainerに、学習対象のモデル、学習用パラメーター、評価用関数、学習用データ、評価用データを設定して初期化します。

# 評価用関数
from sklearn.metrics import accuracy_score

def compute_metrics(result):
    labels = result.label_ids
    preds = result.predictions.argmax(-1)
    acc = accuracy_score(labels, preds)
    return {
        "accuracy": acc,
    }

In [ ]:
# Trainerの設定
from transformers import Trainer, TrainingArguments

# 学習用パラメーター
training_args = TrainingArguments(
    output_dir = "./results",
    num_train_epochs = 2,
    per_device_train_batch_size = 8,
    per_device_eval_batch_size = 32,
    warmup_steps = 500,  # 学習係数が0からこのステップ数で上昇
    weight_decay = 0.01,  # 重みの減衰率
    # evaluate_during_training = True,  # ここの記述はバージョンによっては必要ありません
    logging_dir = "./logs",
)

# Trainerの初期化
trainer = Trainer(
    model = model, # 学習対象のモデル
    args = training_args, # 学習用パラメーター
    compute_metrics = compute_metrics, # 評価用関数
    train_dataset = train_data, # 学習用データ
    eval_dataset = test_data, # テスト用データ
)

In [ ]:
# モデルの学習
# Trainerを使って、モデルの学習、評価を行います。
trainer.train() # 学習
trainer.evaluate() # 評価

In [ ]:
# 学習済みモデルの保存
# 後で使うために、学習済みモデル、トークナイザーを保存します。
model_dir = "./model"
trainer.save_model(model_dir)
tokenizer.save_pretrained(model_dir)

# 3. Test

In [1]:
# では、学習したモデルを使って、ニュースが正しく分類できるかどうかを確認します。
# 今回はsports-watchディレクトリにあるsports-watch-4764756.txtを入力として分類を行います。

import os
import torch
from transformers import BertForSequenceClassification, BertJapaneseTokenizer

# 学習済みモデルの読み込み
model_dir = "./model"
loaded_model = BertForSequenceClassification.from_pretrained(model_dir)
# loaded_model.cuda() # cudaを使う場合は、この行を有効にする
loaded_tokenizer = BertJapaneseTokenizer.from_pretrained(model_dir)

# 分類するデータの読み込み
file = "./text/sports-watch/sports-watch-4764756.txt"  # sports-watchの適当なニュース

with open(file, "r") as f:
  sample_text = f.readlines()[3:]
  sample_text = "".join(sample_text)
  sample_text = sample_text.translate(str.maketrans({"\n":"", "\t":"", "\r":"", "\u3000":""})) 

2022-12-19 22:04:25.346423: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-19 22:04:26.058578: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2022-12-19 22:04:26.058672: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2022-12-19 22:04:26.058679: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nv

In [2]:
max_length = 512
words = loaded_tokenizer.tokenize(sample_text)
word_ids = loaded_tokenizer.convert_tokens_to_ids(words)  # 単語をインデックスに変換
word_tensor = torch.tensor([word_ids[:max_length]])  # Tensorに変換

In [16]:
print(words[:5])
print(f"{len(words)} 語")
print(word_ids[:5])
print(len(word_ids))

['サッカー', 'W', '杯', '南アフリカ', '大会']
341 語
[1301, 472, 3411, 7716, 622]
341


In [6]:
# 予測の実行
word_tensor.cuda()  # cudaを使う場合は、この行を有効にする
y = loaded_model(word_tensor)  # 結果の予測
pred = y[0].argmax(-1)  # 最大値のインデックス（ディレクトリの番号）

# 結果の標準
path = "./text"
dir_files = os.listdir(path=path)
dirs = [f for f in dir_files if os.path.isdir(os.path.join(path, f))]  # ディレクトリ一覧
print("結果は", dirs[pred])

結果は sports-watch
